In [32]:
from pl_bolts.models.self_supervised import SimCLR, SSLFineTuner
import pytorch_lightning as pl
from pl_bolts.models.regression import LogisticRegression
from pl_bolts.datamodules import STL10DataModule
import torch.nn as nn
from torchvision.datasets import STL10
from torchvision import transforms


In [12]:
weight_path = './trained_models/rgb/epoch=49-step=4800.ckpt'
simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)
resnet50 = simclr.encoder
resnet50.eval()

d:\cs198\SIMCLR\simclr\lib\site-packages\pytorch_lightning\core\saving.py:225: UnderReviewWarning: The feature SimCLR is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  obj = cls(**_cls_kwargs)
d:\cs198\SIMCLR\simclr\lib\site-packages\pl_bolts\models\self_supervised\simclr\simclr_module.py:138: UnderReviewWarning: The feature resnet50 is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  return backbone(first_conv=self.first_conv, maxpool1=self.maxpool1, return_all_feature_maps=False)
d:\cs198\SIMCLR\simclr\lib\site-packages\pl_b

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [37]:
STL10_DATA_PATH = './data/'
stl10 = STL10DataModule(data_dir=STL10_DATA_PATH)
input_dim = 3 * 96 * 96

C:\Users\rayno\AppData\Local\Temp\ipykernel_13028\1757463776.py:2: UnderReviewWarning: The feature STL10DataModule is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  stl10 = STL10DataModule(data_dir=STL10_DATA_PATH)


In [29]:
import os
print(os.listdir(STL10_DATA_PATH))

['class_names.txt', 'fold_indices.txt', 'test_X.bin', 'test_y.bin', 'train_X.bin', 'train_y.bin', 'unlabeled_X.bin']


In [15]:
classifier = LogisticRegression(
    input_dim=input_dim,
    num_classes=stl10.num_classes,
    learning_rate=1e-7
)

C:\Users\rayno\AppData\Local\Temp\ipykernel_13028\3595088951.py:1: UnderReviewWarning: The feature LogisticRegression is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  classifier = LogisticRegression(


In [ ]:
class UnFreezeCallBack(pl.Callback):
    def on_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch == 10.:
            encoder.unfreeze()

In [54]:
import torch
from copy import deepcopy
import torch.utils.data as data

from tqdm.notebook import tqdm


In [33]:
img_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [42]:
train_img_data = STL10(root=STL10_DATA_PATH, split="train", download=True, transform=img_transforms)
test_img_data = STL10(root=STL10_DATA_PATH, split="test", download=True, transform=img_transforms)

print("Number of training examples:", len(train_img_data))
print("Number of test examples:", len(test_img_data))

Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 5000
Number of test examples: 8000


In [55]:
device = 'cpu'
NUM_WORKERS=1
@torch.no_grad()
def prepare_data_features(model, dataset):
    # Prepare model
    network = deepcopy(model.)
    network.fc = nn.Identity()  # Removing projection head g(.)
    network.eval()
    network.to(device)

    # Encode all images
    data_loader = data.DataLoader(dataset, batch_size=64, num_workers=NUM_WORKERS, shuffle=False, drop_last=False)
    feats, labels = [], []
    for batch_imgs, batch_labels in tqdm(data_loader):
        batch_imgs = batch_imgs.to(device)
        batch_feats = network(batch_imgs)
        feats.append(batch_feats.detach().cpu())
        labels.append(batch_labels)

    feats = torch.cat(feats, dim=0)
    labels = torch.cat(labels, dim=0)

    # Sort images by labels
    labels, idxs = labels.sort()
    feats = feats[idxs]

    return data.TensorDataset(feats, labels)

In [56]:
train_feats_simclr = prepare_data_features(simclr, train_img_data)
test_feats_simclr = prepare_data_features(simclr, test_img_data)

AttributeError: 'SimCLR' object has no attribute 'convnet'

In [58]:
from pl_bolts.models.self_supervised.simclr \
    import SimCLREvalDataTransform
from pl_bolts.models.self_supervised.simclr.transforms \
    import SimCLRFinetuneTransform

In [63]:
transform = SimCLREvalDataTransform(input_height=96)
train_img_data_trans = transform(train_img_data[0][0])


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [50]:
print(train_img_data)
print(train_img_data[0][0])
for (x,y) in train_img_data:
    print(len(x),y)

Dataset STL10
    Number of datapoints: 5000
    Root location: ./data/
    Split: train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )
tensor([[[ 0.1451,  0.1451,  0.1451,  ...,  0.2706,  0.2706,  0.2706],
         [ 0.1451,  0.1451,  0.1451,  ...,  0.2784,  0.2784,  0.2784],
         [ 0.1451,  0.1451,  0.1451,  ...,  0.2863,  0.2863,  0.2863],
         ...,
         [ 0.0196,  0.0353, -0.0824,  ...,  0.0745,  0.1765,  0.1059],
         [ 0.0510,  0.2157, -0.3098,  ...,  0.0588,  0.0667,  0.0745],
         [ 0.0588, -0.2392, -0.0902,  ...,  0.0824,  0.1529,  0.0824]],

        [[ 0.1216,  0.1216,  0.1216,  ...,  0.2471,  0.2471,  0.2471],
         [ 0.1216,  0.1216,  0.1216,  ...,  0.2392,  0.2392,  0.2392],
         [ 0.1216,  0.1216,  0.1216,  ...,  0.2392,  0.2392,  0.2392],
         ...,
         [-0.1059, -0.0667, -0.1843,  ...,  0.0353,  0.0824, -0.0431],
         [-0.0745,  0.1059, -0.4196,  ..

In [46]:
epochs = 5
for epoch in range(5):
    for (x, y) in train_img_data:
        feats = resnet50(x)
        y_hat = classifier(feats)
        loss = nn.CrossEntropyLoss(y_hat, y)

    if epoch == 10:
        resnet50.unfreeze()

ValueError: expected 4D input (got 3D input)

In [ ]:
finetuner = SSLFineTuner(simclr, in_features=simclr.z)

In [9]:
trainer = pl.Trainer(callbacks=[UnFreezeCallBack()])

trainer.fit(
    classifier,
    stl10.train_dataloader(batch_size=256),
    stl10.val_dataloader(batch_size=256)
)

GPU available: False, used: False


PicklingError: Can't pickle <function _is_device_tpu at 0x000001C9FBA589D0>: it's not the same object as lightning_lite.accelerators.tpu._is_device_tpu